# Recommender - Systems - [Netflix]

## Matrix factorization techniques for recommender systems 
-Yehuda Koren (Yahoo Research), Robert Bell and Chris Volinsky (AT&T Labs - Research)

----------------------------------------
### Recommender system strategies
* the content filtering (콘텐츠 필터링 접근방식)
    * 특성화 할 각 사용자 또는 제품에 대한 프로필을 생성
        * ex) 영화 프로필에는 장르, 참여 배우, 영화에 관한 속성이 포함
        * 프로필이 사용자와 일치하는 제품과 연결 
    * 성공적인 사례 : 인터넷에 사용되는 음악 게놈 프로젝트         
        * 사용자와 가장 적절한 연결은 사용자 만족도와 충성도를 높이는데 중요 
        * 더 많은 업체들이 사용자의 패턴을 분석하는 추천시스템에 관심을 가짐
    
* collaborative filtering (협업 필터링) 
    * 과거 사용자 행동에 의존하는 content filtering의 대안
        * 프로필을 따로 생성할 필요가 없음
        * 사용자들 과 제품간의 상호의존성 관계를 통해 new user-item 관계를 확인함
    * 도메인이 자유로움
        * 프로필 생성에 어려운 데이터를 분석할 수 있다
        * 시스템의 새로운 프로덕트와 유저를 처리할수 없다
    * 협업 필터링의 주요 영역
        * neighborhood methods
            * 아이템,유저간에 관계를 계산하는데 초점
            * 이웃 항목에 대한 사용자의 선호도를 동일한 항목으로 평가
        * latent factor models(잠재요인 모델)
            * 대안적 접근 방식 : 각 아이템과 사용자들을 특성화
            * 사용자가 해당 특성의 영화에서 높은 점수를 받는 것을 좋아한다
            * (현실적,판타지) , (여성적,남성적)
            
###  Matrix factorization methods
* 잠재요인 모델의 성공적인 구현 : matrix factorization
    * 항목과 사용자를 패턴에서 추론해서 특성화, 항목과 사용자 요인 간의 높은 상관도를 가질경우 추천
    * 실제 상황을 모델링하기 위한 유연성을 가짐
* 추천 시스템 
    * 다양한 유형의 입력 데이터에 의존, 사용자를 나타내는 하나의 차원, 아이템을 나타내는 다른 차원의 매트릭스
    * 가장 편리한 데이터 : 명시적 피드백 (사용자의 관심에 대한 명시적 인자)
    * 영화에 대한 스타 등급, 평점 및 선호도
    * 명시적 피드백 : 일반적으로 sparse matrix로 표현됨, 사용자가 모든 아이템에 랭크를 매기지는 않았기 때문
    * matrix factorization의 강점
        * 추가 정보를 통합가능
        * 명시적 피드백을 이용할 수 없을 때 : 암묵적 피드백을 이용해 선호도 유추
            * 구매 이력, 검색 내역, 검색 패턴 및 사용자 행동을 이용해 반영
            * 암묵적 피드백은 대개 사건의 유무를 나타냄 : dense matrix
            
###   A basic matrix factorization model
* matrix factorization models : classic nearest-neighbor보다 추천시스템에 효과적임
    * 사용자와 아이템을 모두 매핑
        * 잠재 요인 공간 f ( 사용자-아이템 상호작용 ) : 내적을 통해 모델링
    * 방정식(1)
        * $\hat r_{ui} = q_i^Tp_u$ $\to$ user $u$의 item $i$에 대한 평점 = item이 갖고 있는 요소 $q_i$ user가 갖고 있는 요소 $p_u$

* SVD(singular value decomposition)
    * 협업 필터링에 적용 : user-item rating matrix를 분해
        * missing value로 인해 어려움
            * 일반적인 SVD는 matrix의 정보가 sparse하면 불확실하고, 상대적으로 존재하는 값에 overfitting될 수 있다
        * 누락된 정보들을 채우기 위해 imputation(귀속화)
            * 데이터의 양이 많을 수록 많은 비용 소모
            * 부정확한 imputation은 데이터를 왜곡
    * 정규화된 모델을 통한 overfitting을 피하면서 관측된 등급만을 직접 모델링
        * $p_u,q_i$를 학습하기 위해 regularized squared error 를 최소화
            * 방정식(2)
                * $min \Sigma (r_{ui} - q_i^Tp_u)^2 + \lambda(\lVert q_i \rVert^2 + \lVert p_u \rVert^2)$
        * 알려지지 않은 미래 rating예측을 위한 이전 rating학습 및 일반화 과정
        * $\lambda$ 상수 : 정규화의 정도를 조정, 일반적으로 cross-validation을 통해 결정
        
### Learning algorithms
* 방정식(2)를 최소화 하는 두가지 접근 방식
    * stochastic gradient descent(SGD)
    * alternating least squares(ALS)

* Stochastic gradient descent(SGD)
    * prediction error($e_{ui} = r_{ui} - q_i^Tp_u$)를 최소화 하는 parmeter를 gradient descent로 계산 
        * $q_i \leftarrow q_i + \gamma \centerdot(e_{ui}\centerdot p_u - \lambda \centerdot q_i)$
        * $p_u \leftarrow p_u + \gamma \centerdot(e_{ui}\centerdot q_i - \lambda \centerdot p_u)$
    * running time이 빠르지만 어떤 경우에는 ALS를 사용하는 것이 유용

* Alternating least squares(ALS)
    * $q_i$와 $p_u$는 unknown하기 때문에 방정식(2)는 convex(볼록)하지 않다.
        * unknown값들 중 하나를 수정하면 최적화 문제가 quadratic하게 되고 최적으로 해결할수 있다
        * ALS techniques  
            * $p_u$를 고정, $q_i$를  수렴할때 까지 계산 
            * $q_i$를 고정, $p_u$를 수렴할때 까지 계산
    * ALS가 SGD보다 유리한 경우
        1. 시스템이 parallelization(병렬화,평행화)을 사용할수 있는 경우
            * $q_i$를 다른 요소들과 독립적으로 계산한 후 $p_u$를 다른 사용자 요소와 독립적으로 계산
            * 잠재적으로 알고리즘의 대규모 병렬화가 가능
        2. 시스템이 명시적 데이터에 중점이 되어있는 경우
            * training set이 sparse한 것을 고려하지 않을때 SGD에 비해 효율이 더 좋다

### Adding biases
* 협업 필터링에 대한 matrix factorization
    * 다양한 데이터 양상과 다른 어플리케이션의 특정 요구조건에 유연하게 대처할수 있음
    * 방정식(1)과 같은 학습형태를 가짐
        * 사용자와 서로 다른 rating 값을 산출하는 아이템과의 상호작용을 찾음
        * 관측된 값의 많은 차이는 상호작용과는 독립된 biases(편향),intercept의 영향
            * ex) 협업 필터링 데이터 : 일부 사용자가 다른 사용자보다 높은 rating을 부여하고 일부 아이템이 다른 것들에 비해 높은 rating을 받는 system적 경향을 가짐
            * 어떤 제품들은 다른 제품들보다 낫다고 인식됨
            * 단순히 $q_i^Tp_u$의 형식의 상호작용에 의해 전체 설명하는데 한계

* 데이터의 실제 상호작용 부분과 요인 모델링만을 조건으로 하여 bias들을 식별
    * 방정식(3) : rating $r_{ui}$에 포함된 bias의 근사치 $b_{ui}$
        *  $b_{ui}=\mu + b_i + b_u$
            * $\mu$: 전체 평균, $b_i,b_u$ : item,user 각각의 편차를 평균으로 부터 나타냄
    * ex) user Joe의 titanic영화의 rating의 first-order estimate
        * 모든 영화에 대한 평균 평점 : $\mu$ = 3.7
        * 타이타닉은 평균보다 0.5점 높게 평가하는 경향이 있다
        * Joe는 비판적인 사용자로 평균보다 0.3개의 별을 낮게 평가하는 경향이 있다
        * 따라서 Joe의 rating : 3.7+0.5-0.3 = 3.9
        * 방정식(4 $\leftarrow$ 1,3) : $\hat r_{ui}=\mu + b_i + b_u + q_i^Tp_u$
        * 관측 등급 : 글로벌 평균 + item_bia + user_bia + user-item_bia
            * 각 속성들은 그것과 관련된 부분만 설명 가능
            * 방정식(5 $\leftarrow$ 4) - minimizing squared error function을 통한 학습
            * min $\Sigma(r_{ui}-\mu-b_u-b_i-p_u^Tq_i)^2
            + \lambda(\lVert p_u\rVert^2+\lVert p_u\rVert^2+b_u^2+b_i^2)$
        * 편향 : 관측된 신호의 대부분을 포착 $\to$ 정확한 모델링이 필수

### Additional input sources
* 콜드 스타트 문제 : rating을 거의 하지 않은 많은 사용자들의 취향에 대한 일반적인 결론을 내리기 어려움
    * solution : 추가적인 user에 대한 정보를 포함시킴
    * 추천자 시스템 : 사용자 선호도에 대한 인사이트를 얻기 위해 암시적 피드백을 사용
        * implicit(암시적) : 명시적의 반대, 간접적인 정보
        * ex) 구매 내역, 검색 내역을 통한 그들의 경향 파악
    * 단순성을 위해 boolean implicit feedback을 고려
        * $N(u)$ : 사용자 u의 암묵적 선호도를 표현한 항목의 집합
            * 시스템이 사용자가 암묵적으로 선호하는 항목을 통해 사용자를 프로파일링
            * 새로운 item i의 암묵적 요인 $x_i$
                * N(u)의 item i에 대한 선호도를 보여주는 유저를 $x_i$vector로 characterize
                * $\Sigma x_i$
                    * Normalizing해서 유용하게 사용
                    * ex) $|N(u)|^{-0.5}\Sigma x_i$
        * $A_u$ : 사용자 u를 설명할 수 있는 속성(ex.인구통계,성별,연령,소득)의 boolean
            * 고유 인자벡터 $y_a$ : 사용자를 설명하는 각 속성
            * $\Sigma y_a$
            * $\hat r_{ui} = \mu + b_i b_u + q_i^T[p_u + |N(u)|^{-0.5}\Sigma x_i + \Sigma y_a]$
            * 데이터 lack이 일반적인 경우에 대처 방안
            
### Temporal dynamics
* 위의 모델들은 모두 정적 : 현실에서는 계속해서 변화함
    * 고객들의 성향도 진화하여 취향을 재정립 하게 해야함
    * 사용자-아이템 상호작용의 동시적인 특성을 반영하는 시간적 효과를 고려

* matrix factoriazation
    * 시간적 영향을 모델링하는데 도움,정확성 향상
    * Decomposing ratings : 구분되는 용어로 분해
        * 시간적 측면을 별도로 취급 가능
            * 아이템 bias : $b_i(t)$, user bias : $b_u$
            * user 선호도 : $p_u(t)$
    * 시간적 영향 1 : 아이템의 인기가 시간이 지남에 따라 바뀜
        * ex) 영화는 배우의 새 영화 출연과 같은 외부사건에 의해 유발되는 인기몰이를 함
        * item bias의 시간 함수로 취급
    * 시간적 영향 2 : 사용자가 시간에 따라 기준 등급을 변경할 수 있게 함
        * 평균적으로 4점을 주는 사용자는 그 이후로 그영화를 3점이라고 생각한다.
        * 사용자의 등급 척도에서 자연적인 변화를 포함한 요인을 반영
            * 사용자는 다른 최근 등급과 비교하여 등급을 부여한다.
        * user bias의 시간 함수로 취급
    * temporal dynamics : 사용자 선호도, user-item간 상호작용의 영향 
        * 사용자들은 시간이 지남에 따라 선호도를 바꿈
        * ex) 범죄드라마 팬이 시간이 지나 스릴러 장르의 팬이 될수 있음, 특정 배우나 감독에 대한 인식이 바뀔 수 있음
        * 모델 : user선호도(pu)을 시간 함수로 취급
        * $q_i$ : 정적인 item 특성. 물건들은 시간이 지나도 변하지 않음
    * Exact parameterizations of time-varying parameters
        * 방정식(7)
            * $\hat r_{ui}(t) = \mu + b_i(t)+b_u(t)+q_i^tp_u(t)$


### Inputs with varying confidence levels
?